In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [48]:
"""
Could you make a Google drive with a giant excel sheet with many tabs. Here are tabs needed (I added columns name in " ", some are quite long):

•	"Hardware mentions" (which should have 14,530 rows right?): 
o	Column 1: "Date mentioned"
o	Column 2: "Link to the thread" (if you have, otherwise no need of this column
o	Column 3: "Hardware name" (put the after-mapping name for consistency)
o	Column 3: "Power efficiency (TH/J)"
o	Column 4: "Release date" (so that's not the date it was mentioned, but the date the hardware company released it)
"""

'\nCould you make a Google drive with a giant excel sheet with many tabs. Here are tabs needed (I added columns name in " ", some are quite long):\n\n•\t"Hardware mentions" (which should have 14,530 rows right?): \no\tColumn 1: "Date mentioned"\no\tColumn 2: "Link to the thread" (if you have, otherwise no need of this column\no\tColumn 3: "Hardware name" (put the after-mapping name for consistency)\no\tColumn 3: "Power efficiency (TH/J)"\no\tColumn 4: "Release date" (so that\'s not the date it was mentioned, but the date the hardware company released it)\n'

In [49]:
# date,hardware_name,TH/J
# 2010-09-09 12:59:39,gtx460,0.0000004270
hardware = pd.read_csv("../bitcoinforum/5_processing_extracted_data/hardware_instances_with_efficiency.csv")
hardware.rename(columns={"date": "Date mentioned", "hardware_name": "Hardware name", "TH/J": "Power efficiency (TH/J)"}, inplace=True)

# \#,Type,Hardware name,Release date,First date used,Eff. (TH/J)
# 1,GPU,8800gts,Feb 2007,2011-03-08,1.09E-07
paper_list = pd.read_csv("../hardwarelist/paper_list.csv")
paper_list["Hardware name"] = paper_list["Hardware name"].str.lower()

# Merge on hardware name to get the release date
df1 = hardware.merge(paper_list[["Hardware name", "Release date"]], on="Hardware name", how="left")
df1.to_csv("csv/Hardware mentions.csv", index=False)
df1

,Date mentioned,Hardware name,Power efficiency (TH/J),Release date
0,2010-09-09 12:59:39,gtx460,4.270000e-07,Jul 2010
1,2010-10-06 20:25:17,4350,3.460000e-07,Jan 2009
2,2010-10-06 20:25:17,5770,1.940100e-06,Oct 2009
3,2010-10-06 20:25:17,5870,1.906000e-06,Sep 2009
4,2010-10-06 20:25:17,gtx260,2.100000e-07,Dec 2009
...,...,...,...,...
14530,2023-12-17 03:45:25,microbt whatsminer m30s,2.631579e-02,Apr 2020
14531,2023-12-17 03:45:25,microbt whatsminer m50s,3.846100e-02,Jul 2022
14532,2023-12-17 03:45:25,microbt whatsminer m50s,3.846100e-02,Jul 2022
14533,2023-12-17 03:45:25,microbt whatsminer m50s,3.846100e-02,Jul 2022


In [50]:
"""
•	"Hardware count by year" 
o	Here you just remake tab S20
"""

'\n•\t"Hardware count by year" \no\tHere you just remake tab S20\n'

In [51]:
df2 = df1[["Date mentioned"]].copy()
df2["Date mentioned"] = df2["Date mentioned"].apply(lambda x: x[:4])
df2 = df2.groupby("Date mentioned").size().reset_index(name="Count")
df2.rename(columns={"Date mentioned": "Year"}, inplace=True)
# remove 2010
df2 = df2[df2["Year"] != "2010"]
df2.to_csv("csv/Hardware count by year.csv", index=False)
df2

,Year,Count
1,2011,1147
2,2012,803
3,2013,2001
4,2014,2379
5,2015,1899
6,2016,1320
7,2017,1569
8,2018,1160
9,2019,575
10,2020,363


In [52]:
"""
•	"Power efficiency" 
o	Column 1: "Quarter" (I guess you did quarterly, so you can write like Q1 2011, Q2 2011....)
o	Column 2: "Average power efficiency (TH/J)" (here you put the average we calculated for each quarter)
o	Column 3: "Max attainable power efficiency (TH/J)" (here you put Pmax value for each quarter, you can put the Pmax value you have for each quarter beginning)
"""

'\n•\t"Power efficiency" \no\tColumn 1: "Quarter" (I guess you did quarterly, so you can write like Q1 2011, Q2 2011....)\no\tColumn 2: "Average power efficiency (TH/J)" (here you put the average we calculated for each quarter)\no\tColumn 3: "Max attainable power efficiency (TH/J)" (here you put Pmax value for each quarter, you can put the Pmax value you have for each quarter beginning)\n'

In [53]:
# date,price,hashrate,coins_per_block,efficiency,max_efficiency
# 2023-12-31,38658.06,5.008642376892796e+20,6.25,34412596500.0,54100000000.0
monthly_stuff = pd.read_csv("../bitcoinforum/6_merging/monthly_stuff.csv")
monthly_stuff["date"] = pd.to_datetime(monthly_stuff["date"])
monthly_stuff["Quarter"] = monthly_stuff["date"].dt.to_period("Q")
monthly_stuff["Quarter"] = monthly_stuff["Quarter"].astype(str)
monthly_stuff = monthly_stuff[["Quarter", "efficiency", "max_efficiency"]]
df3 = monthly_stuff.groupby("Quarter").agg({"efficiency": "mean", "max_efficiency": "first"}).reset_index()
df3.rename(columns={"efficiency": "Average power efficiency (TH/J)", "max_efficiency": "Max attainable power efficiency (TH/J)"}, inplace=True)
df3.to_csv("csv/Power efficiency.csv", index=False)
df3.head(3)

,Quarter,Average power efficiency (TH/J),Max attainable power efficiency (TH/J)
0,2011Q1,1.676753e+06,3780000.0
1,2011Q2,1.662901e+06,3780000.0
2,2011Q3,1.777678e+06,23300000.0


In [54]:
"""
•	"Electricity consumption and carbon footprint"
o	Here you just remake Tab S18
"""

'\n•\t"Electricity consumption and carbon footprint"\no\tHere you just remake Tab S18\n'

In [55]:
# the file is exported by plots/main.ipynb

In [56]:
"""
•	"Bitcoin price and mining costs" 
o	Column 1: "Bitcoin price" each row is a day since Jan 2011, and you put the price everyday until end of December 2023
o	Column 2: "Mining costs with C = 0.03 USD/kWh; PUE = 1.0"
o	Column 3: "Mining costs with C = 0.05 USD/kWh; PUE = 1.1"
o	Column 4: "Mining costs with C = 0.07 USD/kWh; PUE = 1.2"
o	Column 5: "Price-cost ratio with C = 0.03 USD/kWh; PUE = 1.0"
o	Column 6: "Price-cost ratio with C = 0.05 USD/kWh; PUE = 1.1"
o	Column 7: "Price-cost ratio with C = 0.07 USD/kWh; PUE = 1.2"
"""

'\n•\t"Bitcoin price and mining costs" \no\tColumn 1: "Bitcoin price" each row is a day since Jan 2011, and you put the price everyday until end of December 2023\no\tColumn 2: "Mining costs with C = 0.03 USD/kWh; PUE = 1.0"\no\tColumn 3: "Mining costs with C = 0.05 USD/kWh; PUE = 1.1"\no\tColumn 4: "Mining costs with C = 0.07 USD/kWh; PUE = 1.2"\no\tColumn 5: "Price-cost price with C = 0.03 USD/kWh; PUE = 1.0"\no\tColumn 6: "Price-cost price with C = 0.05 USD/kWh; PUE = 1.1"\no\tColumn 7: "Price-cost price with C = 0.07 USD/kWh; PUE = 1.2"\n'

In [66]:
# date,joules_per_coin
# 2023-12-31,1397249022408.723
joules_per_coin = pd.read_csv("joules_per_coin.csv") # exported by plots/main.ipynb
monthly_stuff = pd.read_csv("../bitcoinforum/6_merging/monthly_stuff.csv")
price = monthly_stuff[["date", "price"]].copy()

df4 = price.merge(joules_per_coin, on="date", how="left")
df4["kwh_per_coin"] = df4["joules_per_coin"] / 3600000
df4["Mining costs with C = 0.03 USD/kWh; PUE = 1.0"] = df4["kwh_per_coin"] * 0.03
df4["Mining costs with C = 0.05 USD/kWh; PUE = 1.1"] = df4["kwh_per_coin"] * 0.05 * 1.1
df4["Mining costs with C = 0.07 USD/kWh; PUE = 1.2"] = df4["kwh_per_coin"] * 0.07 * 1.2
df4["Price-cost ratio with C = 0.03 USD/kWh; PUE = 1.0"] = df4["price"] / df4["Mining costs with C = 0.03 USD/kWh; PUE = 1.0"]
df4["Price-cost ratio with C = 0.05 USD/kWh; PUE = 1.1"] = df4["price"] / df4["Mining costs with C = 0.05 USD/kWh; PUE = 1.1"]
df4["Price-cost ratio with C = 0.07 USD/kWh; PUE = 1.2"] = df4["price"] / df4["Mining costs with C = 0.07 USD/kWh; PUE = 1.2"]
df4.rename(columns={"price": "Bitcoin price"}, inplace=True)
df4.drop(columns=["joules_per_coin", "kwh_per_coin"], inplace=True)
df4.to_csv("csv/Bitcoin price and mining costs.csv", index=False)
df4

,date,Bitcoin price,Mining costs with C = 0.03 USD/kWh; PUE = 1.0,Mining costs with C = 0.05 USD/kWh; PUE = 1.1,Mining costs with C = 0.07 USD/kWh; PUE = 1.2,Price-cost ratio with C = 0.03 USD/kWh; PUE = 1.0,Price-cost ratio with C = 0.05 USD/kWh; PUE = 1.1,Price-cost ratio with C = 0.07 USD/kWh; PUE = 1.2
0,2023-12-31,38658.0600,11643.741853,21346.860065,32602.477190,3.320072,1.810948,1.185740
1,2023-10-31,27978.1000,14776.112668,27089.539892,41373.115471,1.893468,1.032801,0.676239
2,2023-09-30,25816.5700,14485.653053,26557.030597,40559.828548,1.782217,0.972118,0.636506
3,2023-08-31,29629.2400,16929.701012,31037.785189,47403.162835,1.750134,0.954618,0.625048
4,2023-07-31,30589.0500,16393.181084,30054.165320,45900.907034,1.865962,1.017797,0.666415
...,...,...,...,...,...,...,...,...
149,2011-06-30,9.5700,0.465775,0.853920,1.304169,20.546422,11.207139,7.338008
150,2011-05-31,3.0331,0.138976,0.254789,0.389133,21.824638,11.904348,7.794514
151,2011-04-30,0.7741,0.042495,0.077908,0.118987,18.216109,9.936059,6.505753
152,2011-03-31,0.9202,0.037427,0.068616,0.104795,24.586600,13.410872,8.780928
